# Домашнее задание 3. 

## Предсказание пользовательской оценки отеля по тексту отзыва.

Мы собрали для вас отзывы по 1500 отелям из совершенно разных уголков мира. Что это за отели - секрет. Вам дан текст отзыва и пользовательская оценка отеля. Ваша задача - научиться предсказывать оценку отеля по отзыву. Данные можно скачать [тут](https://www.kaggle.com/c/hseds-texts-2020/data?select=train.csv).

Для измерения качества вашей модели используйте разбиение данных на train и test и замеряйте качество на тестовой части.

#### Про данные:
Каждое ревью состоит из двух текстов: positive и negative - плюсы и минусы отеля. В столбце score находится оценка пользователя - вещественное число 0 до 10. Вам нужно извлечь признаки из этих текстов и предсказать по ним оценку.

Удачи! 💪

#### Использовать внешние данные для обучения строго запрещено. Можно использовать предобученные модели из torchvision.

In [1]:
import random
import torch
import numpy as np
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [2]:
import string
import numpy as np
import nltk

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [4]:
!unzip /content/drive/MyDrive/hseds-texts-2020.zip > /dev/null 

In [5]:
PATH_TO_TRAIN_DATA = 'train.csv'

In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error

df = pd.read_csv(PATH_TO_TRAIN_DATA)
df.head()

,review_id,negative,positive,score
0,00003c6036f30f590c0ac435efb8739b,There were issues with the wifi connection,No Positive,7.1
1,00004d18f186bf2489590dc415876f73,TV not working,No Positive,7.5
2,0000cf900cbb8667fad33a717e9b1cf4,More pillows,Beautiful room Great location Lovely staff,10.0
3,0000df16edf19e7ad9dd8c5cd6f6925e,Very business,Location,5.4
4,00025e1aa3ac32edb496db49e76bbd00,Rooms could do with a bit of a refurbishment ...,Nice breakfast handy for Victoria train stati...,6.7


Предобработка текста может сказываться на качестве вашей модели.
Сделаем небольшой препроцессинг текстов: удалим знаки препинания, приведем все слова к нижнему регистру. 
Однако можно не ограничиваться этим набором преобразований. Подумайте, что еще можно сделать с текстами, чтобы помочь будущим моделям? Добавьте преобразования, которые могли бы помочь по вашему мнению.

Также мы добавили разбиение текстов на токены. Теперь каждая строка-ревью стала массивом токенов.

### Часть 1. 1 балл

Сначала токенизацируем и лемматизируем, уберем пунктуацию и цифру

In [7]:
def process_text(text):
  wordnet_lemmatizer = WordNetLemmatizer()
  return ' '.join([wordnet_lemmatizer.lemmatize(word) for word in word_tokenize(text.lower()) if word not in string.punctuation])

def minus_digits(text):
  return ' '.join([word for word in text.split(' ') if not any(map(str.isdigit, word))])

In [8]:
df['negative'] = df['negative'].apply(process_text)
df['positive'] = df['positive'].apply(process_text)

In [9]:
df['negative'] = df['negative'].apply(minus_digits)
df['positive'] = df['positive'].apply(minus_digits)

In [10]:
df['final_preprocessing'] = df['negative'] + ' ' + df['positive'] #сделаем столбец где будем хранить негативные и позитивные отзывы вместе
df_train, df_test = train_test_split(df)

In [11]:
sklearn_pipeline = Pipeline((('vect', TfidfVectorizer()),
                             ('cls', SGDRegressor())))
sklearn_pipeline.fit(df_train['final_preprocessing'], df_train['score']);

In [12]:
mean_absolute_error(df_test['score'], sklearn_pipeline.predict(df_test['final_preprocessing']))

0.9744102977331616

### Часть 3. 6 баллов

Теперь давайте воспользуемся более продвинутыми методами обработки текстовых данных, которые мы проходили в нашем курсе. Обучите RNN/Transformer для предсказания пользовательской оценки. Получите ошибку меньше, чем во всех вышеперечисленных методах.

Если будете обучать RNN, попробуйте ограничить максимальную длину предложения. Некоторые отзывы могут быть слишком длинные относительно остальных.

Чтобы пользоваться DataLoader, все его элементы должны быть одинаковой размерности. Для этого вы можете добавить нулевой паддинг ко всем предложениям (см пример pad_sequence)

Для дальнейшей работы будем ипользовать данный сайт https://huggingface.co/transformers/model_doc/bert.html


In [13]:
import torch
from torch import nn
from torch.nn import functional as F

In [14]:
!pip install pytorch_transformers

     |████████████████████████████████| 184kB 13.1MB/s 
     |████████████████████████████████| 133kB 35.0MB/s 
     |████████████████████████████████| 890kB 32.7MB/s 
     |████████████████████████████████| 1.1MB 49.2MB/s 
     |████████████████████████████████| 7.2MB 50.8MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=904b5faf5058158244bab1c7730c7c925e9dcd02a13853caa48abf260e7af461
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.19.41 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [15]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence
from pytorch_transformers import BertTokenizer, BertConfig
from pytorch_transformers import BertForSequenceClassification
from tqdm import tqdm

In [16]:
set_score = list(set(df.score.tolist()))
dict_score = {set_score[item]:item for item in range(len(set_score))}
def change(score):
  return(dict_score[score['score']])
  
df['score_class'] = df.apply(change, axis=1)

df_train, df_test = train_test_split(df)
# создадим score_class в котором будем хранить метки классов для каждого score, и опять разобъем датафрейм на test и train

In [17]:
# Берт принимает предложения тип '[CLS] hi [SEP]'
sentences_train = ["[CLS] " + sentence + " [SEP]" for sentence in df_train['final_preprocessing']]
labels_train = [item for item in df_train['score_class'].tolist()]

sentences_test = ["[CLS] " + sentence + " [SEP]" for sentence in df_test['final_preprocessing']]
labels_test = [item for item in df_test['score_class'].tolist()]

In [18]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) # будем использовать базовые Берт, где слова имеют нижний регистр

tokenized_texts_train = [tokenizer.tokenize(sent) for sent in sentences_train]
print (tokenized_texts_train[0])

100%|██████████| 231508/231508 [00:00<00:00, 36303695.01B/s]


['[CLS]', 'honestly', 'nothing', 'it', 'wa', 'a', 'perfect', 'experience', 'very', 'professional', 'and', 'helpful', 'staff', '[SEP]']


In [19]:
MAX_LEN = 200
train_input = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts_train]

train_pos_pad = pad_sequence([torch.as_tensor(seq[:MAX_LEN]) for seq in train_input], 
                           batch_first=True)

train_masks = [[float(i>0) for i in seq] for seq in train_pos_pad]

Token indices sequence length is longer than the specified maximum sequence length for this model (649 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (513 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (547 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (557 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (582 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

In [20]:
tokenized_texts_test = [tokenizer.tokenize(sent) for sent in sentences_test]

test_input = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts_test]
test_pos_pad = pad_sequence([torch.as_tensor(seq[:MAX_LEN]) for seq in test_input], 
                           batch_first=True)

validation_masks = [[float(i>0) for i in seq] for seq in test_pos_pad]

Token indices sequence length is longer than the specified maximum sequence length for this model (640 > 512). Running this sequence through the model will result in indexing errors


In [22]:
train_inputs = torch.tensor(train_pos_pad)
train_labels = torch.tensor(labels_train)
train_masks = torch.tensor(train_masks)


validation_inputs = torch.tensor(test_pos_pad)
validation_labels = torch.tensor(labels_test)
validation_masks = torch.tensor(validation_masks)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [23]:
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(
    train_data, shuffle=True,
    batch_size=32
)


validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_dataloader = DataLoader(
    validation_data,
    batch_size=32
)

In [24]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=37)
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

100%|██████████| 440473133/440473133 [00:05<00:00, 82307172.01B/s]


In [25]:
# функция перевода метки класса в score
dict_score_class = {item:set_score[item] for item in range(len(set_score))} #set_score = list(set(df.score.tolist())) (Напоминание)
def mae(pred, label):
  pred =  [dict_score_class[item] for item in pred]
  label = [dict_score_class[item] for item in label]
  return mean_absolute_error(pred, label)

In [26]:
def train_one_epoch(model, train_dataloader, optimizer, device="cuda:0"):
    model.to(device).train()
    with tqdm(total=len(train_dataloader)) as pbar:
     for batch in train_dataloader:
        # добавляем батч для вычисления на GPU
        # Распаковываем данные из dataloader
        input_ids, input_mask, labels = batch
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        loss = model(input_ids, token_type_ids=None, attention_mask=input_mask, labels=labels)
        loss[0].backward()
        optimizer.step()

        predicted = loss[1].detach().cpu().numpy()
       
        batch_preds = np.argmax(predicted, axis=1)
        accuracy_mae = mae(batch_preds, labels.cpu().numpy())
        pbar.set_description('Loss: {:.4f}; Accuracy_MAE: {:.4f}'.format(loss[0].item(), accuracy_mae))    
        pbar.update(1)

In [27]:
train_one_epoch(model, train_dataloader, optimizer)

Loss: 2.1259; Accuracy_MAE: 1.0219:  95%|█████████▌| 2234/2344 [42:44<02:06,  1.15s/it]


KeyboardInterrupt: ignored

In [28]:
def predict(model, val_dataloader, device="cuda:0"):
    model.to(device).eval()
    losses = []
    predicted_classes = []
    true_classes = []
    valid_preds, valid_labels = [], []
    with tqdm(total=len(val_dataloader)) as pbar:
      with torch.no_grad():
        for batch in val_dataloader:
          # добавляем батч для вычисления на GPU
          batch = tuple(t.to(device) for t in batch)
    
          # Распаковываем данные из dataloader
          input_ids, input_mask, labels = batch
          
          logits = model(input_ids, token_type_ids=None, attention_mask=input_mask)
          
          logits = logits[0].detach().cpu().numpy()
          label_ids = labels.to('cpu').numpy()
          
          
          batch_preds = np.argmax(logits, axis=1)
    
          batch_labels = np.stack(label_ids)     
          valid_preds.extend(batch_preds)
          valid_labels.extend(batch_labels)

          accuracy_mae = mae(batch_preds, label_ids)
          pbar.set_description('Accuracy_MAE: {:.4f}'.format(accuracy_mae))
          pbar.update(1)

    return valid_preds, valid_labels


In [29]:
predicted_classes, true_classes = predict(model, validation_dataloader)
print('MAE: ', mae(predicted_classes,true_classes ))

Accuracy_MAE: 1.1375: 100%|██████████| 782/782 [05:32<00:00,  2.35it/s]

MAE:  0.987692
